# Exploración, limpieza y obtención de coordenadas

In [ ]:
# Librerías
import numpy as np
import pandas as pd
import ast

from utils.columnas import cargar_datos_activos, contar_y_ultimo_elemento, obt_fecha
from utils.limpieza_direccion import preparar_para_geocoding
from utils.coordenadas import analizar_distancias_por_empleado
from utils import data_utils as du
from obtencion_distancias_ttc import calcular_distancias_transporte_backup

from TTC import TTCReubicacion


import warnings
warnings.filterwarnings("ignore")

## Lectura de conjunto de datos

In [ ]:
# Lectura último conjunto de datos agregado a raw/maestro
dir_maestro = r".\data\raw\maestro"
n_maestro, lista_maestro = contar_y_ultimo_elemento(dir_maestro)
ultimo_maestro = sorted(lista_maestro)[-1]
data_activos = cargar_datos_activos(dir_maestro + "\\" + ultimo_maestro)
tiendas = pd.read_csv(r".\data\processed\tiendas.csv")

In [ ]:
# Se transforman las columnas de Fechas
## Activos
data_activos['FECHA DE INGRESO'] = obt_fecha(data_activos['FECHA DE INGRESO'])
data_activos['FECHA DE NACIMIENTO'] = obt_fecha(data_activos['FECHA DE NACIMIENTO'])

In [ ]:
# Columnas de interés 
columnas_activos = ["NUMERO DE IDENTIFICACION", "LUGAR DE TRABAJO", "PUESTO", "JORNADA", "079_DIRECCION", "CIUDAD RESIDENCIA"]

### Filtro de conjunto de datos

Para el análisis Bogotá se tendrá en cuenta las categorías de la columna CIUDAD DONDE TRABAJA == (SOACHA, CHÍA, BOGOTÁ, MOSQUERA)

#### Analisis de población

In [ ]:
data_activos['LUGAR DE TRABAJO'].value_counts()

In [ ]:
data_activos.describe(include='all')

In [ ]:
len(data_activos[~data_activos['LUGAR DE TRABAJO'].isin(['OFICINA CENTRAL APOYO', 'CENTRO LOGISTICO'])])

In [ ]:
# Grupos a exluir del análisis
excluir_grupo = ["OFICINA_A"]
excluir_ceco = ["Centro Atencion Cliente"]
excluir_lugar = ["OFICINA CENTRAL APOYO", "CENTRO LOGISTICO"]

# Lugares a inclui en el análisis
categorias_bogota = ["BOGOTÁ", "CHÍA", "SOACHA", "MOSQUERA"]

In [ ]:
data_bogota_activos = data_activos[data_activos['CIUDAD DONDE TRABAJA'].isin(categorias_bogota)]
data_bogota_activos = data_bogota_activos[~((data_bogota_activos['LUGAR DE TRABAJO'].isin(excluir_lugar)) | 
                                            (data_bogota_activos['DESCRIPCION CECO'].isin(excluir_ceco)) | 
                                            (data_bogota_activos['GRUPO '].isin(excluir_grupo)) | 
                                            (data_bogota_activos['LUGAR DE TRABAJO'].isna())
                                            )]
data_bogota_activos.shape

### Bogotá Activos

In [ ]:
data_bogota_activos = data_activos[data_activos['CIUDAD DONDE TRABAJA'].isin(categorias_bogota)]
data_bogota_activos = data_bogota_activos[~((data_bogota_activos['LUGAR DE TRABAJO'].isin(excluir_lugar)) | 
                                            (data_bogota_activos['DESCRIPCION CECO SAP'].isin(excluir_ceco)) | 
                                            (data_bogota_activos['GRUPO PLANEACION'].isin(excluir_grupo)) | 
                                            (data_bogota_activos['LUGAR DE TRABAJO'].isna())
                                            )]
data_bogota_activos

In [ ]:
len(data_bogota_activos['LUGAR DE TRABAJO'].unique())

In [ ]:
data_bogota_activos['LUGAR DE TRABAJO'].unique()

## Obtención de coordenadas

#### Activos

##### Limpieza de direcciones

In [ ]:
data_bogota_activos

In [ ]:
data_bogota_activos_direcciones = preparar_para_geocoding(data_bogota_activos, '079_DIRECCION', 'CIUDAD RESIDENCIA')
data_bogota_activos_direcciones

##### Coordenadas

In [ ]:
dir_backup = r".\data\backup_coordenadas\back_up_activos.csv"

# data_bogota_activos_coordenadas = creacion_df_coordendas(data_bogota_activos_direcciones, dir_backup)

In [ ]:
dir_export_activos = r".\data\coordenadas\coordenadas_activos.csv"
data_bogota_activos_coordenadas = pd.read_csv(dir_export_activos)
# data_bogota_activos_coordenadas.to_csv(dir_export_activos, index= False)

In [ ]:
data_bogota_activos_coordenadas

### Obtención de distancias (línea recta)

#### Activos

In [ ]:
# Conjunto de datos con alta precisión de geocodificación
d_b_a_coordenadas_palta = data_bogota_activos_coordenadas[data_bogota_activos_coordenadas['precision'] == 'Alta']

# Se selecciona las tiendas que están en bogotá y alrededores 
ct_bogota = list(set(data_bogota_activos_coordenadas[data_bogota_activos_coordenadas['CIUDAD DONDE TRABAJA'].isin(categorias_bogota)]["LUGAR DE TRABAJO"].unique()))

# Conjunto de datos de tiendas Bogotá y sus coordenadas
tiendas_bogota = tiendas[tiendas['LUGAR DE TRABAJO'].isin(ct_bogota)]

In [ ]:
d_b_a_coordenadas_palta

In [ ]:
# Obtención de distancias
df_direcciones_alta_distancias = d_b_a_coordenadas_palta.join(
    d_b_a_coordenadas_palta.apply(lambda row: analizar_distancias_por_empleado(row, tiendas_bogota.copy()), axis = 1)
)

In [ ]:
df_direcciones_alta_distancias

In [ ]:
df_direcciones_alta_distancias[~df_direcciones_alta_distancias['tiendas_mas_cercanas_que_actual'].isnull()]

In [ ]:
df_direcciones_alta_distancias[df_direcciones_alta_distancias['distancia_tienda_actual'] >= 50]

In [ ]:
du.analizar_columna(df_direcciones_alta_distancias, "distancia_tienda_actual")

In [ ]:
# Conjunto de datos sin datos atípicos 
df_direcciones_alta_distancias_sin_atipicos = df_direcciones_alta_distancias[df_direcciones_alta_distancias['distancia_tienda_actual'] <= 50]

In [ ]:
du.analizar_columna(df_direcciones_alta_distancias_sin_atipicos, "distancia_tienda_actual")

In [ ]:
df_direcciones_alta_distancias[(df_direcciones_alta_distancias['distancia_tienda_actual'] <= 50) & (df_direcciones_alta_distancias['distancia_tienda_actual'] >= 25)]

In [ ]:
df_direcciones_alta_distancias_sin_atipicos

### Medición de distancias con medio de transporte simulado

#### Activos

In [ ]:
df_direcciones_alta_distancias_sin_atipicos["tiendas cercanas"] = df_direcciones_alta_distancias_sin_atipicos['tiendas_mas_cercanas_que_actual'].apply(
    lambda lista: [t[0] for t in lista] if isinstance(lista, list) else []
)

In [ ]:
# df_direcciones_alta_distancias_sin_atipicos.to_csv(r".\data\proccesed\activos_objetivo.csv")

In [ ]:
dir_backup_activos = r".\data\backup_distancias\back_up_activos.csv"
# calcular_distancias_transporte_backup(df_direcciones_alta_distancias_sin_atipicos, tiendas_bogota, dir_backup_activos)

##### Exportación de conjuntos de datos

In [ ]:
# Activos distancias
distancias_activos = pd.read_csv(dir_backup_activos)
# distancias_activos.to_csv(".\data\distancias\distancias_activos.csv")

### Análisis de resultados de tiempos y distancias

##### Activos

In [ ]:
dir_backup_activos = r".\data\backup_distancias\back_up_activos.csv"
distancias_activos = pd.read_csv(dir_backup_activos)
distancias_activos

In [ ]:
du.analizar_columna(distancias_activos, "distancia_actual_km")

In [ ]:
du.analizar_columna(distancias_activos, "tiempo_actual_min")

In [ ]:
distancias_activos[distancias_activos['tiempo_actual_min']>=150]

In [ ]:
# Validación de personas a las cuales se les calculó la distancia y duración en transporte público
distancias_activos[distancias_activos['distancia_actual_km'].notnull()]

In [ ]:
no_distancias_activos = distancias_activos[distancias_activos['distancia_actual_km'].isnull()]
lista_no_distancias_activos = no_distancias_activos['id_persona'].tolist()

In [ ]:
df_direcciones_alta_distancias_sin_atipicos[
    df_direcciones_alta_distancias_sin_atipicos['NUMERO DE IDENTIFICACION'].isin(lista_no_distancias_activos)]['LUGAR DE TRABAJO'].value_counts()

In [ ]:
df_direcciones_alta_distancias_sin_atipicos['LUGAR DE TRABAJO'].value_counts()

In [ ]:
df_modelo_final = df_direcciones_alta_distancias_sin_atipicos
df_modelo_final

In [ ]:
df_modelo_final = df_direcciones_alta_distancias_sin_atipicos.merge(
    distancias_activos, left_on='NUMERO DE IDENTIFICACION', right_on='id_persona', how='left')
df_modelo_final

In [ ]:
columnas_interes = ["NUMERO DE IDENTIFICACION", "EDAD", "GENERO", 
                    "SUELDO", "FECHA DE INGRESO", "LUGAR DE TRABAJO", 
                    "PUESTO", "DESCRIPCION JORNADA", 
                    "FTES ACTUAL", "ESTADO CIVIL", "NIVEL GRI", "distancia_actual_km",
                    "tiempo_actual_min", "distancias_cercanos"]

df_modelo_final = df_modelo_final[columnas_interes]
df_modelo_final


In [ ]:
# Se eliminan los registros sin distancia calculada
df_modelo_final = df_modelo_final[df_modelo_final['distancia_actual_km'].notnull()]
df_modelo_final

In [ ]:
# Se eliminan los registros que hacen parte del almacen mosquera y chia
df_modelo_final = df_modelo_final[~df_modelo_final['LUGAR DE TRABAJO'].isin(['ALMACEN MOSQUERA', 'ALMACEN CHIA'])]
df_modelo_final

#### Implementación Algoritmo TTC (Top Trading Cycles)

Nuestra población objetivo son vendedores con un jornada 100% 

In [ ]:
# Vendedores - Jornada 100%
df_vendedores = df_modelo_final[(df_modelo_final['PUESTO'] == 'VENDEDOR') & (df_modelo_final['DESCRIPCION JORNADA'] == 'JORNADA 100%')]

# Columnas de interés
columnas = ['NUMERO DE IDENTIFICACION', 'LUGAR DE TRABAJO', 'PUESTO',
       'DESCRIPCION JORNADA', 'distancia_actual_km', 'tiempo_actual_min',
       'distancias_cercanos']

df_vendedores = df_vendedores[columnas]

# tipo de dato de la columna 'distancias_cercanos'
df_vendedores['distancias_cercanos'] = df_vendedores['distancias_cercanos'].apply(ast.literal_eval)

# Conjunto de datos vendedores con tiendas más cercanas a la actual
df_vendedores_reubi = df_vendedores[df_vendedores['distancias_cercanos'].map(len) > 0]

# Se convierte la columna 'distancias_cercanos' a tipo str para implementar el algoritmo TTC
df_vendedores_reubi['distancias_cercanos'] = df_vendedores_reubi['distancias_cercanos'].astype('str')

# Se implementa algoritmo TTC
ttc = TTCReubicacion(df_vendedores_reubi)
df_resultado, df_ciclos = ttc.ejecutar(verbose=True)

In [ ]:
df_resultado

#### Resultados

In [ ]:
vendedores_sin_reubicacion = df_vendedores[~df_vendedores['NUMERO DE IDENTIFICACION'].isin(list(df_resultado['NUMERO DE IDENTIFICACION']))]

# Asigna los valores requeridos a las columnas faltantes en vendedores_sin_reubicacion
vendedores_sin_reubicacion['ciclo'] = np.nan
vendedores_sin_reubicacion['distancia_recomendada_km'] = vendedores_sin_reubicacion['distancia_actual_km']
vendedores_sin_reubicacion['tiempo_recomendado_min'] = vendedores_sin_reubicacion['tiempo_actual_min']
vendedores_sin_reubicacion['mejora_%_distancia'] = 0

# concatenar conjuntos de datos
df_resultados = pd.concat([df_resultado, vendedores_sin_reubicacion], ignore_index=True)

In [ ]:
# Total de colaboradores analizados
total_colaboradores_analizados = df_resultados.shape[0]

# Total de colaboradores con posible reubicación
colaboradores_posible_reubi = df_vendedores_reubi.shape[0]

# Total de colaborares reubicados
colaboradores_reubi = len(df_resultado[df_resultado['LUGAR DE TRABAJO'] != df_resultado['tienda_recomendada']])

# Distancia promedio actual
distancia_promedio_actual = df_resultados['distancia_actual_km'].mean()

# Distancia promedio de posible reubicación
distancia_promedio_posible_reubi = df_vendedores_reubi['distancia_actual_km'].mean()

# Distancia promedio después de reubicación posibles reubicaciones
distancia_promedio_despues_reubi_posibles = df_resultado['distancia_recomendada_km'].mean()

# Distancia promedio después de reubicación general
distancia_promedio_despues_reubi = df_resultados['distancia_recomendada_km'].mean()

# Tiempo promedio actual
tiempo_promedio_actual = df_resultados['tiempo_actual_min'].mean()

# Tiempo promedio de posible reubicación
tiempo_promedio_posible_reubi = df_vendedores_reubi['tiempo_actual_min'].mean()

# Tiempo promedio después de reubicación posibles reubicaciones
tiempo_promedio_despues_reubi_posibles = df_resultado['tiempo_recomendado_min'].mean()

# Tiempo promedio después de reubicación general
tiempo_promedio_despues_reubi = df_resultados['tiempo_recomendado_min'].mean()



In [ ]:
print("Total de colaboradores analizados:", total_colaboradores_analizados)
print("Total de colaboradores con posible reubicación:", colaboradores_posible_reubi)
print("Total de colaboradores reubicados:", colaboradores_reubi)

print("Distancia promedio actual:", distancia_promedio_actual)
print("Distancia promedio de posible reubicación:", distancia_promedio_posible_reubi)
print("Distancia promedio después de reubicación (posibles):", distancia_promedio_despues_reubi_posibles)
print("Distancia promedio después de reubicación (general):", distancia_promedio_despues_reubi)

print("Tiempo promedio actual:", tiempo_promedio_actual)
print("Tiempo promedio de posible reubicación:", tiempo_promedio_posible_reubi)
print("Tiempo promedio después de reubicación (posibles):", tiempo_promedio_despues_reubi_posibles)
print("Tiempo promedio después de reubicación (general):", tiempo_promedio_despues_reubi)


In [ ]:
# Resultados porcentuales
porcentaje_personas_posible_reubi = (colaboradores_posible_reubi / total_colaboradores_analizados) * 100
porcentaje_personas_reubicadas = (colaboradores_reubi / colaboradores_posible_reubi) * 100
mejora_promedio_distancia_general = ((distancia_promedio_actual - distancia_promedio_despues_reubi) / distancia_promedio_actual) * 100
mejora_promedio_distancia_posibles = ((distancia_promedio_posible_reubi - distancia_promedio_despues_reubi_posibles) / distancia_promedio_posible_reubi) * 100
mejora_tiempo_promedio_general = ((tiempo_promedio_actual - tiempo_promedio_despues_reubi) / tiempo_promedio_actual) * 100
mejora_tiempo_promedio_posibles = ((tiempo_promedio_posible_reubi - tiempo_promedio_despues_reubi_posibles) / tiempo_promedio_posible_reubi) * 100


In [ ]:
print("Porcentaje de personas con posible reubicación: {:.2f}%".format(porcentaje_personas_posible_reubi))
print("Porcentaje de personas reubicadas: {:.2f}%".format(porcentaje_personas_reubicadas))
print("Mejora promedio de distancia (general): {:.2f}%".format(mejora_promedio_distancia_general))
print("Mejora promedio de distancia (posibles): {:.2f}%".format(mejora_promedio_distancia_posibles))
print("Mejora promedio de tiempo (general): {:.2f}%".format(mejora_tiempo_promedio_general))
print("Mejora promedio de tiempo (posibles): {:.2f}%".format(mejora_tiempo_promedio_posibles))